In [ ]:
!pip install -q langchain transformers PyPDF2 sentence-transformers install pinecone-client xformers bitsandbytes accelerate

In [ ]:
import pandas as pd
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline, PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import os
import pinecone

In [ ]:
#mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Read in HP troubleshooting pdf
hp_guide=PdfReader('/content/gdrive/MyDrive/Colab Notebooks/Langchain/HP_project/HP_troubleshooting.pdf')

In [ ]:
#Extract all of the text from the pdf
hp_guide_text=''
for i, page in enumerate(hp_guide.pages):
  text=page.extract_text()
  if text:
    hp_guide_text+=text



In [ ]:
#Split the text into chunks for use in ChatGPT
text_splitter=RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=200)
split_text=text_splitter.split_text(hp_guide_text)
print("Number of text chunks: {}  \n".format(len(split_text)))

print("Example of text_chunk \n")
split_text[0]


Number of text chunks: 284  

Example of text_chunk 



'Troubleshooting Guide www.hp.com 1–11\nComputer Diagnostic Features\nDiagnostics for Windows\nThe Diagnostics for Windows (DFW)  utility allows you to view \ninformation about the hardware and software configuration of the computer while running Microsoft Windows XP. It also allows you to perform hardware and software te sts on the subsystems of the \ncomputer.\nWhen you invoke Diagnostics for Wi ndows, the Overview screen is'

In [ ]:
#Instantiating Sentence transformers embeddings to use for Pinecone index
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
#initializing pinecone environement
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', '')
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)



In [ ]:
#creating Pinecone index - Commented out so that index is not recreated with each notebook run
#hp_embeddings_indexed=Pinecone.from_texts(split_text, embeddings, index_name="hptroubleshooting")

In [ ]:
#downloading already created index from Pinecone
hp_embeddings_indexed = Pinecone.from_existing_index("hptroubleshooting", embeddings)

##Downloading LLama 2 Model/Tokenizer and creating a pipeline

In [ ]:
#Downloading optimizer
tokenizer= AutoTokenizer.from_pretrained("DimitryK/Llama-2-7b-chat-hf-HP-v4")

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [ ]:
#Downloading the Llama-2 model
model=AutoModelForCausalLM.from_pretrained("DimitryK/Llama-2-7b-chat-hf-HP-v4",
                                           torch_dtype=torch.float16,
                                           load_in_8bit=True)

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
#creating Huggingface pipeline
pipeline=pipeline("text-generation",
                  model=model,
                  tokenizer=tokenizer,
                  torch_dtype=torch.float16,
                  max_new_tokens=500,
                  do_sample=True,
                  top_p=0.3

                  )

In [ ]:
#creating Langchain Wrapper for Hugging Face pipeline
llm = HuggingFacePipeline(
    pipeline=pipeline,
    model_kwargs={"temperature": 0.1}
)

##LangChain Question and Answer Retrieval From Pinecone Indexed Document

In [ ]:
# System prompt with message to not make up answers
system_prompt = """You are a helpful and honest assistant. Use the context provided to answer the question.
If you don't know the answer, just say that you don't know. Do not make up an answer. Do not repeat yourself over and over.
"""

#User question and reference to the retrieved information
info_and_user_message = """
Context: {context}

Question: {question}
"""
#Prompt template formatted to Llama 2 specification
llama_prompt_template=f"<s> [INST] <<SYS>>\n {system_prompt} \n<</SYS>>\n\n {info_and_user_message } [/INST]"

In [ ]:
#Langchian prompt template with prompt above
prompt = PromptTemplate(template=llama_prompt_template, input_variables=["context", "question"])

In [ ]:
#Q/A retrieval chain with prompt above and retriever to get the best 3 matches from the Pinecone database index
retrieval_chain = RetrievalQA.from_chain_type(llm=llm,
                                            chain_type="stuff",
                                            retriever=hp_embeddings_indexed.as_retriever(search_kwargs={"k":3}),

                                            chain_type_kwargs={"prompt": prompt}

                                            )

In [ ]:
#function to print output
def print_output(output):
    # Print the query
    print("Query: {}\n".format(output['query']))
    output["result"]=output['result'].replace("\\n\n", " ")
    words = output["result"].split()
    lines = [' '.join(words[i:i+25]) for i in range(0, len(words), 25)]
    # Print the result
    print("Result: {} ".format('\n'.join(lines)))

In [ ]:
#Question 1
result = retrieval_chain("Hard drive boot has been disabled in Computer Setup. What is the solution?")
print_output(result)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Query: Hard drive boot has been disabled in Computer Setup. What is the solution?

Result: The solution is: Run Computer Setup and enable hard drive entry in the Boot Order list. If using a multi-hard drive configuration, make sure the
hard drive is listed as 0. If the hard drive boots cannot be detected, then run Computer Setup and disable Minimal Operating Systems and then
re–enable it. If the hard drive boots to DOS, then run Computer Set–– then disable Minimal Operating Systems and then re– enable it. If the
hard drive boots to an operating system other than Windows, then run Computer Setup then disable Minimal Operating Systems and then re– enable it. If
the hard drive still boots to DOS, then run Computer Setup then enable––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

In [ ]:
#Question 2
result_2 = retrieval_chain("What is the purpose of the CMOS and how is it reset?")
print_output(result_2)

Query: What is the purpose of the CMOS and how is it reset?

Result: The CMOS stores password information and computer configuration. To reset the CMOS, you must first access the Computer Setup Utilities menu, then press the F10
key and press Enter to bypass the title screen, if necessary. Then, locate, press, and hold the CMOS button in for five seconds. Make sure
to disconnect the AC power cord from the wall outlet and hold the CMOS button until it beeps. If necessary, see the Computer Setup (F10)
Utility Guide on the Documentation CD for information on backing up CMOS settings before erasing them. Then, allow the computer to search for and find
the CMOS settings. If the CMOS settings are not found, then re-enter the CMOS settings, and allow the computer to search again. If the CMOS
settings are still not found, then restart the computer. 


In [ ]:
#Question 3
result_3 = retrieval_chain("How do you install Diagnostics for Windows on Windows XP?")
print_output(result_3)

Query: How do you install Diagnostics for Windows on Windows XP?

Result: To install Diagnostics for Windows on Windows XP, follow these steps: 1. Close any Windows applications. 2. Install the Diagnostics for Windows utility: ❏In Windows
XP, select Start > Software Setup icon. Select Diagnostics for Windows > Next button. ❏In Windows XP, select Start > Software Setup icon. Select Diagnostics
for Windows > Next button. 3. Click Next to install Diagnostics for Windows. 4. After the program has finished installing, you may be prompted to
install the Diagnostics for Windows SoftPaq. 5. If you want to upgrade an existing version of Diagnostics for Windows installed on the computer, visit www.hp.com/support
to locate and download the latest Diagnostics for Windows SoftPaq. 6. Follow the on-screen instructions to complete the installation. Note: If you do not install
Diagnostics for Windows, some features may not function properly. 
